In [1]:
import csv
from datetime import timedelta
import gurobipy as gp 
from gurobipy import GRB
import networkx as nx
import random
import matplotlib.pyplot as plt
import inspect
from collections import defaultdict

from models import simple_mpc, mpc_duration_constr, lazy, column_generation
from helper import Service, hhmm2mins, mins2hhmm, fetch_data, draw_graph_with_edges, node_legal, no_overlap, create_duty_graph, extract_nodes, generate_paths, roster_statistics, solution_verify, restricted_linear_program, generate_initial_feasible_duties_random_from_services, generate_new_column, mip


In [2]:
services, service_dict = fetch_data('./StepBackServices.csv')
graph = create_duty_graph(services)

### Main

In [3]:
repeat =True
while repeat:
    num = int(input("""\nEnter the Model you'd like to run (integer): 
                    
                    1. Simple MPC
                    2. MPC with Duration Constraint
                    3. Lazy
                    4. Column Generation
                    
                    Expecting input: """))
    if num == 1:
        print("\nSimple MPC Model")
        duties, duty_count = simple_mpc(graph, service_dict, show_logs = False, show_duties = False, show_roster_stats = True)
        # print(duties)
        # print(duty_count)
    elif num == 2:
        print("\nMPC with Duration Constraints")
        duties, duty_count = mpc_duration_constr(graph, service_dict, time_limit = 60, show_logs = True, show_duties = False, show_roster_stats = True)
        # print(duties)
        # print(duty_count)
    elif num == 3:
        print("\nLazy Model")
        duties, duty_count = lazy(graph, service_dict, show_logs = False, max_duty_duration=6*60, lazy_iterations =100, show_lazy_updates_every = 10, show_duties = False, show_roster_stats = True)
    elif num == 4:
        print("\nColumn Generation Model")
        # duties, selected_duties, obj = column_generation(graph, service_dict, init_column_generator = "random", pricing_method = "bellman ford", iterations = 10, verbose = True)
        mpc_sol, column_pool, duties, selected_duties, obj = column_generation(graph, service_dict, init_column_generator = "mpc", mpc_timeout = 300,pricing_method = "topological sort", iterations = 1000, verbose = True)
        roster_statistics(duties.values(), service_dict)
    else:
        print("\nInvalid Input")

    repeat =False

    # repeat = input("\nWould you like to run another model? (y/n): ").lower() == 'y'



Column Generation Model
Set parameter Username
Set parameter LicenseID to value 2619491
Academic license - for non-commercial use only - expires 2026-02-07
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 4600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  300

Optimize a model with 104364 rows, 68020 columns and 335422 nonzeros
Model fingerprint: 0x4522e447
Variable types: 34478 continuous, 33542 integer (33542 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 4061 rows and 1230 columns
Presolve time: 1.33s
Presolved: 100303 rows, 66790 columns, 373298 nonzeros
Variable types: 33424 continuous, 33366 integer (33366 binary)
Found heuristic 

In [4]:
sorted_duties = sorted(selected_duties, key=lambda x: x[1], reverse=True)

print(sorted_duties)

[('x136', 1.0), ('x138', 1.0), ('x141', 1.0), ('x404', 1.0), ('x465', 1.0), ('x538', 1.0), ('x886', 1.0), ('x983', 1.0), ('x38', 0.9136230375095482), ('x55', 0.9128751795168647), ('x47', 0.9030521484621596), ('x33', 0.9019169750113171), ('x11', 0.8989835791434958), ('x30', 0.8829467151098367), ('x74', 0.861153751394159), ('x108', 0.8403629989008038), ('x59', 0.8247718666344944), ('x9', 0.8051965360860671), ('x21', 0.8022754242626772), ('x127', 0.7996731508765772), ('x84', 0.792591081290141), ('x23', 0.7881051131693507), ('x1261', 0.7477537793896398), ('x5', 0.7451576462894638), ('x109', 0.743255816414312), ('x91', 0.738422836915107), ('x24', 0.735868008649773), ('x97', 0.7351135666123096), ('x29', 0.7177462948088751), ('x123', 0.7141585691652831), ('x85', 0.7136163099275661), ('x80', 0.7002442209825093), ('x6', 0.6752428445753361), ('x102', 0.6707767121014747), ('x26', 0.6635123516600725), ('x78', 0.6589598434480548), ('x51', 0.6531833880528515), ('x1271', 0.6424163953271727), ('x90', 

In [5]:
final = []
for var_name, var_val in sorted_duties:
    final.append(duties[var_name])
    if solution_verify(service_dict.values(), final, verbose =False):
        break

print("Required duties: ", len(final))
print("Total duties: ", len(selected_duties))
roster_statistics(final, service_dict)

Required duties:  264
Total duties:  758

Roster Statistics:
Number of duties:  264
Maximum number of services in a duty:  19
Minimum number of services in a duty:  0
Maximum duration of duty:  06:00
Minimum duration of duty:  00:20
Duties with driving time more than 6hrs:  0
Duties with driving time more than 8hrs:  0


In [6]:
obj, selected_duties = mip(service_dict, column_pool, show_solutions = True, show_objective = True, warm = mpc_sol)

Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 4600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  600

Optimize a model with 934 rows, 2076 columns and 14210 nonzeros
Model fingerprint: 0x81ec0da3
Variable types: 0 continuous, 2076 integer (2076 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start produced solution with objective 142 (0.01s)
Loaded user MIP start with objective 142

Presolve removed 9 rows and 312 columns
Presolve time: 0.01s
Presolved: 925 rows, 1764 columns, 13356 nonzeros
Variable types: 0 continuous, 1764 integer (1764 binary)

Root relaxation: objective 1.310450e+02, 5710 iterations, 0.93 seconds (1.45 work units)

    N

ValueError: too many values to unpack (expected 2)

In [ ]:
final_2 = []
for var_name, var_val in selected_duties:
    final_2.append(duties[var_name])
    

print("Required duties: ", len(final))
print("Total duties: ", len(selected_duties))
roster_statistics(final, service_dict)

KeyError: 'x84'